In [1]:
import sys
import os

# add the parent directory (one level up) to sys.path
# sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../maxtext')))

# ! pip install -r ../../maxtext/requirements.txt

!pip uninstall -q -y qwix
!pip install -q git+https://github.com/google/qwix@24294e2e1816fffaee70987023f8f710a27966c2

import MaxText as mt
from MaxText import pyconfig

2025-08-02 22:17:54.582047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754173074.594491   34124 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754173074.598170   34124 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754173074.608649   34124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754173074.608662   34124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754173074.608664   34124 computation_placer.cc:177] computation placer alr

In [2]:
# from MaxText.integration.tunix.tunix_adaptor import TunixMaxTextLlama
# from flax import linen as nn

def get_ref_maxtext_model():

  #python3 -m MaxText.train MaxText/configs/base.yml base_output_directory=${BASE_OUTPUT_DIRECTORY} dataset_path=${DATASET_PATH} tokenizer_path=assets/tokenizer.gemma load_parameters_path=${CONVERTED_CHECKPOINT} per_device_batch_size=1 run_name=${FINETUNE_RUN_NAME} max_target_length=8192 steps=10 async_checkpointing=false model_name=gemma-2b checkpoint_period=5

  #TODO: @mazumdera: change this to use Gemma2-2b-it
  config = pyconfig.initialize(
      ["", "MaxText/configs/base.yml"], #TODO: @mazumdera: why decode.py?
      base_output_directory="gs://dummy_output_dir",  # This is not used in Tunix.
      run_name="test-tunix-maxtext-llama3-8b",
      # dataset_path=we use Tunix's dataset
      # load_parameters_path="gs://maxtext-gemma/2b/", #TODO: @mazumdera: change this to use checkpoint
      # tokenizer_type="tiktoken",
      # tokenizer_path="assets/tokenizer_llama3.tiktoken",
      tokenizer_path="../maxtext/assets/tokenizer.gemma",
      per_device_batch_size=8,
      max_target_length=8192,
      steps=10,
      async_checkpointing="false",
      # model_name="llama3.1-8b",
      model_name="gemma-2b",
      checkpoint_period=5,
      skip_jax_distributed_system="true",
      weight_dtype="bfloat16",
      attention="dot_product"

  )
  model = mt.from_pretrained(
      config)
  
  mesh = model.mesh
  
  return model, mesh, config
#   # checkpoint = mt.checkpointing.load_params_from_path(
#   #     load_parameters_from_path="gs://maxtext-gemma/2b/",
#   #     abstract_unboxed_params=None,
#   #     checkpoint_storage_concurrent_gb=None,
#   # )
#   checkpoint = {}

#   def create_model(config):
#     return mt.from_pretrained(config, rngs=nnx.Rngs(params=0, dropout=1))

#   model = nnx.eval_shape(create_model, config=config)

#   @nnx.jit
#   def partial_init(checkpoint, config):
#     model = create_model(config)
#     nnx.update(model, checkpoint)
#     # shard model
#     state = nnx.state(model)
#     specs = nnx.get_partition_spec(state)
#     state = jax.lax.with_sharding_constraint(state, specs)
#     nnx.update(model, state)
#     return model

#   with jax.sharding.use_mesh(model.mesh), nn.logical_axis_rules(config.logical_axis_rules):
#     model = partial_init(checkpoint, config)
#   print(model)

  
#   tunix_model = TunixMaxTextLlama(
#         base_model=model,
#         use_attention_mask=False,  # trust Tunix loss masking
#     )
#   mesh  = tunix_model.base.mesh
  
#   #TODO: @mazumdera: change this to use llama3.1-8b
#   # model_config = None
#   # We can continue to use Tunix's model_config
#   model_config = gemma_lib.TransformerConfig.gemma2_2b()

#   # Add these lines to properly get the graph definition and state
#   graphdef, state = nnx.split(tunix_model)
#   tunix_model = nnx.merge(graphdef, state)  # Recreate model in proper NNX format
    
  
#   return tunix_model, mesh, model_config

In [3]:
gemma, mesh, config = get_ref_maxtext_model()

Updating keys from env and command line: ['run_name', 'model_name', 'async_checkpointing', 'checkpoint_period', 'weight_dtype', 'attention', 'base_output_directory', 'tokenizer_path', 'per_device_batch_size', 'steps', 'skip_jax_distributed_system', 'max_target_length']
Running Model: gemma-2b
Updating following parameters in config

base_emb_dim: 2048
base_num_query_heads: 8
base_num_kv_heads: 1
base_mlp_dim: 16384
base_num_decoder_layers: 18
head_dim: 256
mlp_activations: ['gelu', 'linear']
vocab_size: 256128
decoder_block: gemma
normalization_layer_epsilon: 1e-06
logits_via_embedding: True
Updating keys from model: ['base_emb_dim', 'base_num_query_heads', 'base_num_kv_heads', 'base_mlp_dim', 'base_num_decoder_layers', 'head_dim', 'mlp_activations', 'vocab_size', 'decoder_block', 'normalization_layer_epsilon', 'logits_via_embedding']
Skipping jax distributed system due to skip_jax_distributed_system=True flag.


Not using emergency checkpoint, ignoring local_checkpoint_directory, local_checkpoint_period, use_replicator_service and replicator_backup_interval_minutes
dataset_type set to tfds, will use keys['dataset_path']='' and keys['dataset_name']='c4/en:3.0.1'
Config param activations_in_float32: False
Config param adam_b1: 0.9
Config param adam_b2: 0.95
Config param adam_eps: 1e-08
Config param adam_eps_root: 0.0
Config param adam_weight_decay: 0.1
Config param add_bos: True
Config param add_eos: True
Config param allow_split_physical_axes: False
Config param ar_cache_axis_order: 1,2,0,3
Config param async_checkpointing: False
Config param attention: dot_product
Config param attention_type: global
Config param attn_logits_soft_cap: None
Config param autoregressive_decode_assert: 
Config param base_emb_dim: 2048
Config param base_mlp_dim: 16384
Config param base_moe_mlp_dim: 7168
Config param base_num_decoder_layers: 18
Config param base_num_kv_heads: 1
Config param base_num_query_heads: 8
Co

In [4]:
gemma

Transformer(
    # attributes
    config = <MaxText.pyconfig.HyperParameters object at 0x78494c0af8b0>
    mesh = Mesh(device_ids=array([[[[[[[[[[[[0]]]]]]]]],
    
    
    
    
    
    
    
    
             [[[[[[[[[1]]]]]]]]],
    
    
    
    
    
    
    
    
             [[[[[[[[[2]]]]]]]]],
    
    
    
    
    
    
    
    
             [[[[[[[[[3]]]]]]]]],
    
    
    
    
    
    
    
    
             [[[[[[[[[7]]]]]]]]],
    
    
    
    
    
    
    
    
             [[[[[[[[[6]]]]]]]]],
    
    
    
    
    
    
    
    
             [[[[[[[[[5]]]]]]]]],
    
    
    
    
    
    
    
    
             [[[[[[[[[4]]]]]]]]]]]]), axis_names=('data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'context', 'context_autoregressive', 'tensor', 'tensor_transpose', 'tensor_sequence', 'expert', 'autoregressive'), axis_types=(Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto))
    quant = None
)

In [5]:
import jax.numpy as jnp
import os
import MaxText as mt
from MaxText import pyconfig
from MaxText import maxtext_utils
import numpy as np
from MaxText.input_pipeline import _input_pipeline_utils
import os
from MaxText.globals import PKG_DIR
from MaxText import max_logging
from MaxText import common_types
import jax
from MaxText import inference_utils

TEMP_BATCH_SIZE = 8

tokens = [2,  49688 ,   736 ,  1280 ,  6987, 235292  ,  108 ,  2299, 235303, 235256,
    573 ,  8957 ,  3646, 235336  ,  108]
repeated_tokens = jnp.repeat(jnp.array(tokens)[None, :], TEMP_BATCH_SIZE, axis=0)
positions = jnp.repeat(jnp.arange(0, len(tokens))[None, :], TEMP_BATCH_SIZE, axis=0)

init_rng = jax.random.PRNGKey(config.init_weights_seed)
state, _ = maxtext_utils.setup_decode_state(gemma, config, init_rng, mesh, None)

full_train_logits = gemma.apply(
          state.params,
            decoder_input_tokens=repeated_tokens,
            decoder_positions=positions,
          decoder_segment_ids = None,
          enable_dropout=False,
          rngs={"aqt": init_rng},
      )
full_train_logits = jax.experimental.multihost_utils.process_allgather(full_train_logits)
max_logging.log(f"{full_train_logits[0, 0, :]=}")


print("Successfully ran the model!")

print(f"Tokens: {tokens}")
print(f"positions: {positions}")


/home/mazumdera_google_com/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No decode checkpoint specified - generating random weights.
No existing checkpoints found, not restoring checkpoint.


full_train_logits[0, 0, :]=array([[-0.94921875,  0.61328125,  9.5625    , ...,  0.74609375,
         0.33789062,  0.11328125],
       [-0.7109375 ,  0.2734375 ,  0.90625   , ..., -1.21875   ,
         0.6171875 , -0.9375    ],
       [ 0.12695312,  0.47851562,  0.86328125, ..., -1.1640625 ,
         0.12158203, -0.9296875 ],
       ...,
       [-0.33203125,  0.921875  ,  0.91796875, ..., -1.390625  ,
         0.8046875 ,  0.09716797],
       [ 0.05932617,  0.328125  , -0.05517578, ..., -0.91796875,
        -0.01245117, -0.41210938],
       [ 0.5625    , -0.04663086, -0.42773438, ..., -1.46875   ,
         0.37304688,  0.38476562]], dtype=float32)
Successfully ran the model!
Tokens: [2, 49688, 736, 1280, 6987, 235292, 108, 2299, 235303, 235256, 573, 8957, 3646, 235336, 108]
positions: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3

In [6]:
full_train_logits.shape

(1, 8, 15, 256128)

In [ ]:
last_token_logits = full_train_logits[:,-1, :, :]
predicted_token_id = jnp.argmax(last_token_logits, axis=-1)
# Decode the token ID to see the predicted word.
# Since TEMP_BATCH_SIZE is 1, we can just grab the first element.
next_token_id = predicted_token_id[0]
# predicted_token_text = gemma_tokenizer.decode([int(next_token_id)])

print(f"\nPredicted next token ID: {next_token_id}")   
# print(f"Predicted next token: '{predicted_token_text}'")


Predicted next token ID: [     2  49688    736   1280   6987 235292    108   2299  78327 235256
    573   8957   3646 235336    108]
